In [1]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from numpy import *

from ggplot import *
from scipy.stats import logistic
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import *
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
import graphviz
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


from linearmodels.iv import IV2SLS
from scipy import stats
from scipy.stats import trim_mean #trimmed mean
from scipy.stats import *

import keyword

#open api 불러올 때 필요한 것
from urllib.request import urlopen #request package 설치
from bs4 import BeautifulSoup #bs4 package 설치

# 그래프에서 한글 깨지지 않기 위헤 사용함
from matplotlib import rcParams
from matplotlib import font_manager,rc

rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='Malgun Gothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)

import FinanceDataReader as fdr
import plotly.graph_objects as go
import plotly.subplots as ms
import math

# Ln7
## 5 최대예상손실액 구하기 이어서...

- 6개월 간의 보유 수익률 = 125개의 daily return 모두 더하면됨
> 로그 수익률 쓰는 이유
- 범위가 대칭적임 +무한대~-무한대 가능
- 일반 수익률은 +무한대~-100%까지만됨
- 가산성
- 필참

In [2]:
df=pd.read_csv('c:/temp/samsung_LN7.csv',index_col='Date',parse_dates=True)
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2010-01-04,16060,16180,16000,16180,239271,0.012516
2010-01-05,16520,16580,16300,16440,559219,0.016069
2010-01-06,16580,16820,16520,16820,459755,0.023114
2010-01-07,16820,16820,16260,16260,443237,-0.033294
2010-01-08,16400,16420,16120,16420,295798,0.009840


In [3]:
df.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2024-04-01,83200,83300,82000,82000,20116513,-0.004854
2024-04-02,82900,85000,82900,85000,37077944,0.036585
2024-04-03,84300,85000,83500,84100,30493347,-0.010588
2024-04-04,85200,85500,84300,85300,25248934,0.014269
2024-04-05,84500,85000,83800,84500,18773002,-0.009379


In [4]:
# 10년 동안의 데이터로 재구성한다.
df1=df.loc['2012-01-01':'2021-12-31']
df1.tail()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-12-24,80200,80800,80200,80500,12086380,0.007509
2021-12-27,80600,80600,79800,80200,10783368,-0.003727
2021-12-28,80200,80400,79700,80300,18226325,0.001247
2021-12-29,80200,80200,78500,78800,19794795,-0.018680
2021-12-30,78900,79500,78100,78300,14236700,-0.006345


In [5]:
# 로그 수익률 변수 만들기
# 로그 수익률: log(p_t)-log(p_(t-1))
# shift(1): 바로 앞의 값
df1['ret']=np.log(df1['Close'])-np.log(df1['Close'].shift(1))
df1.tail()

,Open,High,Low,Close,Volume,Change,ret
Date,,,,,,,
2021-12-24,80200,80800,80200,80500,12086380,0.007509,0.007481
2021-12-27,80600,80600,79800,80200,10783368,-0.003727,-0.003734
2021-12-28,80200,80400,79700,80300,18226325,0.001247,0.001246
2021-12-29,80200,80200,78500,78800,19794795,-0.018680,-0.018857
2021-12-30,78900,79500,78100,78300,14236700,-0.006345,-0.006365


In [7]:
# ret: daily log return

# df2: df1에서 결측치를 삭제한 것
df2=df1.dropna()
df3=df2['ret'] # df2에서 로그 수익률만 가져온 것
df3.tail()

Date
2021-12-24    0.007481
2021-12-27   -0.003734
2021-12-28    0.001246
2021-12-29   -0.018857
2021-12-30   -0.006365
Name: ret, dtype: float64

In [8]:
len(df3)

2462

In [9]:
# 10년간 로그 수익률 데이터가 2462개 있구나

# 2462개 데이터 중에서 무작위로 125개 꺼낸다. 그리고 다 더한다. 
# 이 작업을 여러번 해서 분포를 그린다.
# loop로 이 작업을 1000번을 한다.

sum_boots=[] # 1000개의 6개월 보유수익률이 sum_boots에 쌓인다
days=125
n=10000
random.seed(1234)
# replace=True : 무작위 복원추출
for i in range(n):
    sample=np.random.choice(df3,size=days, replace=True)
    sample_sum=sample.sum() # 125개 로그수익률을 모두 더한 값
    sum_boots.append(sample_sum)
len(sum_boots)

10000

In [10]:
sum_boots

[-0.2366975220549996,
 -0.003454634374318033,
 0.1664111127788246,
 0.1973446375562009,
 0.12858070502292307,
 0.36786925714303287,
 0.05271471619396628,
 0.45631847837967854,
 -0.10843272145197069,
 0.16237044094093633,
 0.09295995006319835,
 0.1752237607217566,
 0.11299092645068853,
 0.029871624966123278,
 -0.20898121910940048,
 -0.0023756660344655955,
 0.37060527869049054,
 0.08687433732883854,
 0.1591371310858012,
 0.1340799251259579,
 0.11662193852086311,
 -0.24224297166751185,
 -0.10985449769928124,
 0.1301790554038842,
 -0.10637033991471867,
 -0.15107352127383145,
 -0.038960340239293956,
 0.2973130405861184,
 0.10512471691381009,
 0.0442726691242985,
 -0.21766122159959878,
 -0.0020370010019536977,
 0.15313982155618255,
 -0.0147127630856847,
 -0.2025860039042744,
 0.10264352844280111,
 0.12880716931912417,
 0.09848129821824081,
 0.05942135559094197,
 -0.008803567270106782,
 -0.09044345199500192,
 0.3551304583573405,
 -0.05178078561132793,
 0.103878207405117,
 -0.01660698224243262

In [11]:
# 첫번째 무작위로 뽑은 결과에 따르면  6개월 보유 수익률이 -23%

# 구해야하는 것: 이 1000개의 값 중에서 5%의 확률인 마이너스 수익률은?
# np의 percentile 활용

ret5=np.percentile(sum_boots,5)
ret5

-0.23921964553444858

In [12]:
# 5%의 작은확률로 -23% 수익률 난다
# 손실금액: 1억*23% = 2400만원

# 90%의 VaR
# ret6=np.percentile(sum_boots,10)
# ret6

# LN8 가구주 직업별 소득 차이(1)

## 1 가계소득 데이터: 가계금융복지조사

In [14]:
df=pd.read_excel('c:/temp/HOUSEHOLD2020.xlsx')
df.head()

,기준년도,MD제공용_가구고유번호,가중값,수도권여부,가구주_성별코드,가구원수,노인가구코드,조손가구코드,한부모가구코드,다문화가구(2012년~),...,원리금상환기인가계지출감소항목코드,부채상환가능시기코드,가구주_은퇴여부,가구주_미은퇴_은퇴연령,가구주_미은퇴_최소생활비,가구주_미은퇴_적정생활비,가구주_미은퇴_노후준비상황코드,가구주_은퇴실제연령,가구주_은퇴_적정생활비충당여부,가구주은퇴_생활비재원마련코드(2014년~)
0,2020,10000112,1531.602344,G1,1,3,G2,G2,G2,G2,...,NaN,1.0,1,70.0,400,500,3.0,NaN,NaN,NaN
1,2020,10000132,1314.119926,G1,2,2,G2,G2,G2,G2,...,4.0,1.0,1,70.0,500,700,3.0,NaN,NaN,NaN
2,2020,10000162,1098.806050,G1,2,1,G1,G2,G2,G2,...,NaN,NaN,1,75.0,100,150,3.0,NaN,NaN,NaN
3,2020,10000182,1124.027871,G1,1,2,G2,G2,G2,G2,...,NaN,NaN,1,70.0,200,250,4.0,NaN,NaN,NaN
4,2020,10000192,1098.806050,G1,2,1,G2,G2,G2,G2,...,NaN,NaN,1,65.0,250,300,3.0,NaN,NaN,NaN


In [16]:
# 변수의 이름이 한글이다. 이러면 잘 못불러옴 파이썬에서는...
# csv로는 안부러와진다
df_csv=pd.read_csv('c:/temp/HOUSEHOLD2020.xlsx')
df_csv.head()

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte

In [19]:
df_csv=pd.read_csv('c:/temp/HOUSEHOLD2020.csv',encoding='CP949')
df_csv.head()

,기준년도,MD제공용_가구고유번호,가중값,수도권여부,가구주_성별코드,가구원수,노인가구코드,조손가구코드,한부모가구코드,다문화가구(2012년~),...,원리금상환기인가계지출감소항목코드,부채상환가능시기코드,가구주_은퇴여부,가구주_미은퇴_은퇴연령,가구주_미은퇴_최소생활비,가구주_미은퇴_적정생활비,가구주_미은퇴_노후준비상황코드,가구주_은퇴실제연령,가구주_은퇴_적정생활비충당여부,가구주은퇴_생활비재원마련코드(2014년~)
0,2020,10000112,1531.602344,G1,1,3,G2,G2,G2,G2,...,NaN,1.0,1,70.0,400,500,3.0,NaN,NaN,NaN
1,2020,10000132,1314.119926,G1,2,2,G2,G2,G2,G2,...,4.0,1.0,1,70.0,500,700,3.0,NaN,NaN,NaN
2,2020,10000162,1098.806050,G1,2,1,G1,G2,G2,G2,...,NaN,NaN,1,75.0,100,150,3.0,NaN,NaN,NaN
3,2020,10000182,1124.027871,G1,1,2,G2,G2,G2,G2,...,NaN,NaN,1,70.0,200,250,4.0,NaN,NaN,NaN
4,2020,10000192,1098.806050,G1,2,1,G2,G2,G2,G2,...,NaN,NaN,1,65.0,250,300,3.0,NaN,NaN,NaN


In [20]:
df.columns

Index(['기준년도', 'MD제공용_가구고유번호', '가중값', '수도권여부', '가구주_성별코드', '가구원수', '노인가구코드',
       '조손가구코드', '한부모가구코드', '다문화가구(2012년~)',
       ...
       '원리금상환기인가계지출감소항목코드', '부채상환가능시기코드', '가구주_은퇴여부', '가구주_미은퇴_은퇴연령',
       '가구주_미은퇴_최소생활비', '가구주_미은퇴_적정생활비', '가구주_미은퇴_노후준비상황코드', '가구주_은퇴실제연령',
       '가구주_은퇴_적정생활비충당여부', '가구주은퇴_생활비재원마련코드(2014년~)'],
      dtype='object', length=153)

In [21]:
# 칼럼 엄청 많음. 다 한국어임.

columns=['기준년도','수도권여부','가구주_성별코드','가구원수','가구주_만연령','경상소득(보완)','가구주_직업대분류코드']

In [22]:
df1=df[columns]
df1.head()

,기준년도,수도권여부,가구주_성별코드,가구원수,가구주_만연령,경상소득(보완),가구주_직업대분류코드
0,2020,G1,1,3,34,6593,3
1,2020,G1,2,2,45,17720,2
2,2020,G1,2,1,73,908,4
3,2020,G1,1,2,58,2748,5
4,2020,G1,2,1,27,1015,2


In [23]:
# 7개 변수 중에서 문자 변수는 무엇인가요?
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18064 entries, 0 to 18063
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   기준년도         18064 non-null  int64 
 1   수도권여부        18064 non-null  object
 2   가구주_성별코드     18064 non-null  int64 
 3   가구원수         18064 non-null  int64 
 4   가구주_만연령      18064 non-null  int64 
 5   경상소득(보완)     18064 non-null  int64 
 6   가구주_직업대분류코드  13214 non-null  object
dtypes: int64(5), object(2)
memory usage: 988.0+ KB


In [24]:
# object: 문자변수 -> 문자변수는 2개 있다
